In [1]:
#import library
import nltk
import string
import re
import pandas as pd
import numpy as np

#import stopword
from nltk.corpus import stopwords 
stopwords_indonesia = stopwords.words('indonesian')
 
#import sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#tokenize
from nltk.tokenize import TweetTokenizer


In [2]:
#read data
def load_data():
    data = pd.read_csv('mandalika21032022_edit1_clean.csv')
    return data

#Pembuatan dataframe
df_twit = load_data()

#menampilkan data teratas
df_twit.head(10)
#len(df_twit)

,teks,label
0,b'Hujan deras tidak menyurutkan semangat ribua...,positif
1,b'Terima kasih mbak atas tread pencerahannya t...,positif
2,b'yang sibuk ngurusin hal2 diluar isi nya mema...,positif
3,b'Pada tahun 1997 mengenalkan ' Pawang Hujan ...,positif
4,b'Menkumham menyambut dan mendukung pelaksanaa...,positif
5,b'Pembalap Indonesia yang turun di ajang Moto3...,positif
6,b'Presiden mengapresiasi seluruh pihak baik pe...,positif
7,b'Semoga pariwisata di NTB kembali menggeliat ...,positif
8,b'Pawang Hujan Sebuah ThreadnnPertamina MotoGP...,positif
9,b'Miris banget yg gak beli tiket gak disana da...,negatif


In [3]:
#definisi dataframe
df = pd.DataFrame(df_twit[['teks']])

#menampilkan dataframe
df.head(10)

,teks
0,b'Hujan deras tidak menyurutkan semangat ribua...
1,b'Terima kasih mbak atas tread pencerahannya t...
2,b'yang sibuk ngurusin hal2 diluar isi nya mema...
3,b'Pada tahun 1997 mengenalkan ' Pawang Hujan ...
4,b'Menkumham menyambut dan mendukung pelaksanaa...
5,b'Pembalap Indonesia yang turun di ajang Moto3...
6,b'Presiden mengapresiasi seluruh pihak baik pe...
7,b'Semoga pariwisata di NTB kembali menggeliat ...
8,b'Pawang Hujan Sebuah ThreadnnPertamina MotoGP...
9,b'Miris banget yg gak beli tiket gak disana da...


In [4]:
#menghilangkan mention/user
def remove_pattern(tweet, pattern):
    r = re.findall(pattern, tweet)
    for i in r:
        tweet = re.sub(i, '', tweet)
    return tweet    
df['remove_user'] = np.vectorize(remove_pattern)(df['teks'], "@[\w]*")

In [5]:

def tweet_clean(tweet):
    #remove angka
    tweet = re.sub('[0-9]+', '', tweet)
    
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
 
    # remove old style retweet text "RT"
    tweet = re.sub(r'RT :[\s]+', '', tweet)
    
     # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    
    #remove coma
    tweet = re.sub(r',','',tweet)
    
     # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    
    #Happy Emoticons
    emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
    
    #Sad Emoticons
    emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
 
    #all emoticons (happy + sad)
    emoticons = emoticons_happy.union(emoticons_sad)
              
    #tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
    
    tweets_clean = []    
    for word in tweet_tokens:
        if (word not in stopwords_indonesia and # remove stopwords
              word not in emoticons and # remove emoticons
                word not in string.punctuation): # remove punctuation
            #tweets_clean.append(word)
            stem_word = stemmer.stem(word) #stemming word
            tweets_clean.append(stem_word)
    return tweets_clean
df['tweet_clean'] = df['remove_user'].apply(lambda x: tweet_clean(x))

In [6]:
df.head(10)

,teks,remove_user,tweet_clean
0,b'Hujan deras tidak menyurutkan semangat ribua...,b'Hujan deras tidak menyurutkan semangat ribua...,"[b hujan, deras, surut, semangat, ribu, tonton..."
1,b'Terima kasih mbak atas tread pencerahannya t...,b'Terima kasih mbak atas tread pencerahannya t...,"[b terima, kasih, mbak, tread, cerah, kait]"
2,b'yang sibuk ngurusin hal2 diluar isi nya mema...,b'yang sibuk ngurusin hal2 diluar isi nya mema...,"[b yang, sibuk, ngurusin, luar, isi, nya, tont..."
3,b'Pada tahun 1997 mengenalkan ' Pawang Hujan ...,b'Pada tahun 1997 mengenalkan ' Pawang Hujan ...,"[b pada, kenal, pawang, hujan, dunia, lagu, ju..."
4,b'Menkumham menyambut dan mendukung pelaksanaa...,b'Menkumham menyambut dan mendukung pelaksanaa...,"[b menkumham, sambut, dukung, laksana, event, ..."
5,b'Pembalap Indonesia yang turun di ajang Moto3...,b'Pembalap Indonesia yang turun di ajang Moto3...,"[b balap, indonesia, turun, ajang, moto, mario..."
6,b'Presiden mengapresiasi seluruh pihak baik pe...,b'Presiden mengapresiasi seluruh pihak baik pe...,"[b presiden, apresiasi, selenggara, kerja, sia..."
7,b'Semoga pariwisata di NTB kembali menggeliat ...,b'Semoga pariwisata di NTB kembali menggeliat ...,"[b moga, pariwisata, ntb, geliat, ekonomi, mas..."
8,b'Pawang Hujan Sebuah ThreadnnPertamina MotoGP...,b'Pawang Hujan Sebuah ThreadnnPertamina MotoGP...,"[b pawang, hujan, threadnnpertamina, motogp, m..."
9,b'Miris banget yg gak beli tiket gak disana da...,b'Miris banget yg gak beli tiket gak disana da...,"[b miris, banget, yg, gak, beli, tiket, gak, s..."


In [7]:
#remove punct
def remove_punct(text):
    text  = " ".join([char for char in text if char not in string.punctuation])
    return text
df['Tweet'] = df['tweet_clean'].apply(lambda x: remove_punct(x))

In [8]:
df.head(10)

,teks,remove_user,tweet_clean,Tweet
0,b'Hujan deras tidak menyurutkan semangat ribua...,b'Hujan deras tidak menyurutkan semangat ribua...,"[b hujan, deras, surut, semangat, ribu, tonton...",b hujan deras surut semangat ribu tonton tribu...
1,b'Terima kasih mbak atas tread pencerahannya t...,b'Terima kasih mbak atas tread pencerahannya t...,"[b terima, kasih, mbak, tread, cerah, kait]",b terima kasih mbak tread cerah kait
2,b'yang sibuk ngurusin hal2 diluar isi nya mema...,b'yang sibuk ngurusin hal2 diluar isi nya mema...,"[b yang, sibuk, ngurusin, luar, isi, nya, tont...",b yang sibuk ngurusin luar isi nya tonton musi...
3,b'Pada tahun 1997 mengenalkan ' Pawang Hujan ...,b'Pada tahun 1997 mengenalkan ' Pawang Hujan ...,"[b pada, kenal, pawang, hujan, dunia, lagu, ju...",b pada kenal pawang hujan dunia lagu judul pluies
4,b'Menkumham menyambut dan mendukung pelaksanaa...,b'Menkumham menyambut dan mendukung pelaksanaa...,"[b menkumham, sambut, dukung, laksana, event, ...",b menkumham sambut dukung laksana event intern...
5,b'Pembalap Indonesia yang turun di ajang Moto3...,b'Pembalap Indonesia yang turun di ajang Moto3...,"[b balap, indonesia, turun, ajang, moto, mario...",b balap indonesia turun ajang moto mario suryo...
6,b'Presiden mengapresiasi seluruh pihak baik pe...,b'Presiden mengapresiasi seluruh pihak baik pe...,"[b presiden, apresiasi, selenggara, kerja, sia...",b presiden apresiasi selenggara kerja siang ma...
7,b'Semoga pariwisata di NTB kembali menggeliat ...,b'Semoga pariwisata di NTB kembali menggeliat ...,"[b moga, pariwisata, ntb, geliat, ekonomi, mas...",b moga pariwisata ntb geliat ekonomi masyaraka...
8,b'Pawang Hujan Sebuah ThreadnnPertamina MotoGP...,b'Pawang Hujan Sebuah ThreadnnPertamina MotoGP...,"[b pawang, hujan, threadnnpertamina, motogp, m...",b pawang hujan threadnnpertamina motogp mandal...
9,b'Miris banget yg gak beli tiket gak disana da...,b'Miris banget yg gak beli tiket gak disana da...,"[b miris, banget, yg, gak, beli, tiket, gak, s...",b miris banget yg gak beli tiket gak sana kome...


In [9]:
df.sort_values('Tweet', inplace = True)
df.drop(df.columns[[0,1]], axis = 1, inplace = True)
df.drop_duplicates(subset ='Tweet', keep = 'first', inplace = True)
df.to_csv('mandalika21032022_edit1_cleanbobot.csv',encoding='utf8', index=False)
df.head(10)

,tweet_clean,Tweet
106,[b],b
283,"[b aakk, tuju, bgt, lgsg, jatuh, hati, xfxfxxx...",b aakk tuju bgt lgsg jatuh hati xfxfxxxfxfxxxf...
1359,"[b acara, motogp, mandalika, ubah, cek, update...",b acara motogp mandalika ubah cek update motog...
1048,[b ada],b ada
104,"[b ada, aja, fqnn]",b ada aja fqnn
32,"[b ada, data, tarik, statistik, juara, dunia, ...",b ada data tarik statistik juara dunia motogp ...
1346,"[b ada, kenal, baju, putih, senggol, dongnn]",b ada kenal baju putih senggol dongnn
952,"[b ada, nonton]",b ada nonton
668,"[b ada, pawang]",b ada pawang
244,"[b ada, pawang, gesnn]",b ada pawang gesnn
